In [103]:
from tqdm.auto import tqdm
import time
import subprocess
import random

def dist(n):
    nums = []
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            nums.append(i * j)
            
    nums = sorted(nums)
    last = -1
    ans = []
    while len(nums):
        num = nums.pop()
        if num != last:
            ans.append(num)
        last = num
        
    return len(ans)

In [104]:
wa = False
n = 1

class Test():
    def __init__(self):
        self.generate_test()
        
    def generate_test(self): 
        global n
        with open("test.in", 'w') as test:
            self.n = n
            self.k = dist(n)
            test.write(f"{self.n} {self.k}\n")
            n += 1
        
    def check_solutions(self, a_out, b_out):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a_out, open(b_out, 'r') as b_out:
            # better check brut 
            self.test_data_brut = []
            for _ in range(self.k):
                x = int(a_out.readline().strip())
                self.test_data_brut.append(x)
                
            
            self.test_data_chad = []
            for _ in range(self.k):
                x = int(b_out.readline().strip())
                self.test_data_chad.append(x)
                
            self.better_check()
    
    def better_check(self):
        global wa
        if self.test_data_brut != self.test_data_chad:
            print("WA")
            wa = True
            return

In [105]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        compile_command = ['g++', program, '-o', exe_name]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(200)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./{self.exe_b}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"{self.exe_a}.out", f"{self.exe_b}.out")
            if wa == True:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [ ]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

 61%|██████    | 122/200 [00:25<00:46,  1.68it/s]

In [ ]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
0,2.676796,0.036216
